In [1]:
#import python packages
import os
import json
import glob
import pandas as pd
import cassandra
import numpy as np
import csv
from prettytable import PrettyTable

In [2]:
#check your current working directory
print(os.getcwd())

/home/srivathsan/harini/Udacity-Data-Engineering/Data_Modeling/Project_2


In [3]:
filepath = os.getcwd() + "/event_data"
print(filepath)

/home/srivathsan/harini/Udacity-Data-Engineering/Data_Modeling/Project_2/event_data


In [5]:
#Join the filepath & roots with subdirectories using glob
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root, '*'))
    print (file_path_list)

['/home/srivathsan/harini/Udacity-Data-Engineering/Data_Modeling/Project_2/event_data/2018-11-25-events.csv', '/home/srivathsan/harini/Udacity-Data-Engineering/Data_Modeling/Project_2/event_data/2018-11-24-events.csv', '/home/srivathsan/harini/Udacity-Data-Engineering/Data_Modeling/Project_2/event_data/2018-11-26-events.csv', '/home/srivathsan/harini/Udacity-Data-Engineering/Data_Modeling/Project_2/event_data/2018-11-15-events.csv', '/home/srivathsan/harini/Udacity-Data-Engineering/Data_Modeling/Project_2/event_data/2018-11-18-events.csv', '/home/srivathsan/harini/Udacity-Data-Engineering/Data_Modeling/Project_2/event_data/2018-11-13-events.csv', '/home/srivathsan/harini/Udacity-Data-Engineering/Data_Modeling/Project_2/event_data/2018-11-03-events.csv', '/home/srivathsan/harini/Udacity-Data-Engineering/Data_Modeling/Project_2/event_data/2018-11-21-events.csv', '/home/srivathsan/harini/Udacity-Data-Engineering/Data_Modeling/Project_2/event_data/2018-11-08-events.csv', '/home/srivathsan/

#Process the files to create data file.csv that will be used by apache cassandra tables

In [6]:
#initiate an empty list for each file
full_data_rows_list = []

#iterate through the file_path_list
for f in file_path_list:
    
#reading the csv file
    with open(f, 'r', encoding = 'utf8', newline = '') as csvfile:
    # creating a csv reader
      csvreader = csv.reader(csvfile)
      next(csvreader)
#extracting each data row one by one & append it
      for line in csvreader:
        #print(line)
        full_data_rows_list.append(line)






In [7]:
print(len(full_data_rows_list))

8056


In [8]:
#print(full_data_rows_list)

#creating a smaller event data file for inserting data into apache cassandra tables
csv.register_dialect('myDialect', quoting = csv.QUOTE_ALL, skipinitialspace= True)


with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline = '') as f: 
    writer = csv.writer(f, dialect = 'myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    
    for row in full_data_rows_list:
        if(row[0] == ''):
          continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
    
        
    
    

In [90]:
#check the num of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8', newline = '') as f:
    print(sum(1 for line in f))

6821


In [91]:
#Conneting to the database
from cassandra.cluster import Cluster
try:
     cluster = Cluster(['127.0.0.1'])
     session = cluster.connect()   
except Exception as e:
     print(e)

In [92]:
#create a key space
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}""")
    
except Exception as e:
    print(e)

In [93]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

# Query 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
From the above requirement, it is clear that there are 2 identifiers i.e composite/compund primary key, 
session ID - partition key & item in session - clustering column. 

In [94]:
#Select query
select_query = """select session_id, item_in_session, artist, song_title, song_length from song_info_by_session
                WHERE session_id = %s
                AND item_in_session = %s""" 

#create a table partitioned by session Id and data in every partition sorted by item in session

In [95]:
#Create table song_info_by_session for the above query
query = "CREATE TABLE IF NOT EXISTS song_info_by_session "
query = query + "(session_id int, item_in_session int, artist text, song_title text, song_length float,  PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)



In [96]:
#Ingesting data from event_datafile_new.csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_info_by_session(session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [97]:
#1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
select_query = """select session_id, item_in_session, artist, song_title, song_length from song_info_by_session
                WHERE session_id = %s
                AND item_in_session = %s""" 

In [99]:
#select statement to verify the data
try:
    rows = session.execute(select_query, (338,4))
except exception as e:
    print(e)

for row in rows:
    print("Session ID: "+str(row.session_id), "\nItem In Session: "+str(row.item_in_session), "\nArtist: "+row.artist, "\nSong Title: "+row.song_title, "\nSong Length: "+str(row.song_length))
    

Session ID: 338 
Item In Session: 4 
Artist: Faithless 
Song Title: Music Matters (Mark Knight Dub) 
Song Length: 495.30731201171875


 # Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
create a table with user id as partition key & data clustered by session id & item in session to maintain     uniqueness in every partition
select artist, song_title, item_in_session, user_first_name, user_last_name, from song_info_by_user
WHERE user_id = 10 AND session_id = 182
  

In [100]:
#Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
query1 = "CREATE TABLE IF NOT EXISTS song_info_by_user "
query1 = query1 + "(user_id int, session_id int, item_in_session int, user_first_name text, user_last_name text, artist text, song_title text, PRIMARY KEY (user_id, session_id, item_in_session))"
try:
    session.execute(query1)
except Exception as e:
    print(e)

In [108]:
#Ingesting data from event_datafile_new.csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_info_by_user(user_id, session_id, item_in_session, user_first_name, user_last_name, artist, song_title)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[0], line[9]))

In [114]:
#Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
select_query1 = """select artist, song_title, item_in_session, user_first_name, user_last_name, from song_info_by_user
                WHERE user_id = %s
                AND session_id = %s""" 

In [123]:
#select statement to verify the data
try:
    rows = session.execute(select_query1, (10,182))
except exception as e:
    print(e)

u = PrettyTable([ 'Item in session', 'Artist', 'Song Title', 'First Name', 'Last Name',  ]) 

for row in rows:
    u.add_row([row.item_in_session, row.artist,row.song_title, row.user_first_name, row.user_last_name])
print(u)
    

+-----------------+-------------------+------------------------------------------------------+------------+-----------+
| Item in session |       Artist      |                      Song Title                      | First Name | Last Name |
+-----------------+-------------------+------------------------------------------------------+------------+-----------+
|        0        |  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|        1        |    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
|        2        | Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|        3        |   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-----------------+-------------------+------------------------------------------------------+------------+-----------+


# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
song title is the partition key and user id is the clustering key. The table has to answer all unique users who listeted to a particular song

In [124]:

query = "CREATE TABLE IF NOT EXISTS user_info_by_song "
query = query + "( song_title text, user_id int, user_first_name text, user_last_name text, PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)


In [125]:
#Ingesting data from event_datafile_new.csv
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_info_by_song(song_title, user_id, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [126]:
##Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
select_query3 = "select user_id, user_first_name, user_last_name, song_title from user_info_by_song WHERE song_title = %s;"
print(select_query3)              

select user_id, user_first_name, user_last_name, song_title from user_info_by_song WHERE song_title = %s;


In [127]:
#select statement to verify the data
try:
    rows = session.execute(select_query3, ('All Hands Against His Own',))
except exception as e:
    print(e)

for row in rows:
    print("User Id: "+ str(row.user_id),
          "\nFirst Name :  "+str(row.user_first_name),
           "\nLast Name: "+str(row.user_last_name),
           "\nSong Title: "+str(row.song_title))
    

User Id: 29 
First Name :  Jacqueline 
Last Name: Lynch 
Song Title: All Hands Against His Own
User Id: 80 
First Name :  Tegan 
Last Name: Levine 
Song Title: All Hands Against His Own
User Id: 95 
First Name :  Sara 
Last Name: Johnson 
Song Title: All Hands Against His Own


In [128]:


query = "drop table song_info_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query2 = "drop table song_info_by_user"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)   
    
query3 = "drop table user_info_by_song"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)     
    

In [9]:
session.shutdown()
cluster.shutdown()

NameError: name 'session' is not defined